# MDL DB BUILD!
This notebook is building the roll damping database

In [ ]:
from jupyterthemes import jtplot
jtplot.style(theme='chesterish', context='notebook', ticks=True, grid=False)

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
import pandas as pd
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt

import data
import copy
from mdldb.mdl_db import MDLDataBase
from mdldb.tables import Base, Model, LoadingCondition, Run, RolldecayLinear, RolldecayDirect
from mdldb.tables import RolldecayNorwegian, RolldecayDirectImproved, RolldecayLinearAnalytical
from mdldb.tables import RolldecayCubic,RolldecayQuadraticBandC,RolldecayQuadraticB,RolldecayLinearB,RolldecaySimplifiedIkeda, RolldecaySimplifiedIkedaUnlimited
from mdldb import mdl_to_evaluation
from evaluation.run_dynamic import RunDynamic
from evaluation.run_manoeuvring import RunZigZag

from rolldecayestimators.direct_estimator import DirectEstimator
from rolldecayestimators.direct_estimator_improved import DirectEstimatorImproved
from rolldecayestimators.direct_estimator_cubic import EstimatorCubic,EstimatorQuadraticB,EstimatorQuadraticBandC,EstimatorLinear
from rolldecayestimators.direct_linear_estimator import DirectLinearEstimator
from rolldecayestimators.norwegian_estimator import NorwegianEstimator
from rolldecayestimators.analytical_linear_estimator import AnalyticalLinearEstimator
from rolldecayestimators.ikeda_estimator import IkedaQuadraticEstimator, IkedaEstimatorFitError
from rolldecayestimators.simplified_ikeda import SimplifiedIkedaInputError
from rolldecayestimators.estimator import FitError

from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
#from rolldecay.equations_lambdify import calculate_acceleration, calculate_velocity
from sklearn.pipeline import Pipeline

import signal_lab.mdl_to_evaluation


In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///' + data.mdl_db_path)
db = MDLDataBase(engine=engine)

In [ ]:
#lowpass_filter = LowpassFilterDerivatorTransformer(cutoff=0.4, minimum_score=0.0)
lowpass_filter = LowpassFilterDerivatorTransformer(cutoff=2, minimum_score=0.99)
scaler = ScaleFactorTransformer(scale_factor=None)  # dummy value None for now
cutter = CutTransformer(phi_max=np.deg2rad(9), phi_min=np.deg2rad(0.25), phi1d_start_tolerance=0.015)
offset_transformer = OffsetTransformer()

## Linear method

In [ ]:
direct_linear_estimator = DirectLinearEstimator()

steps = [
    ('filter',lowpass_filter),
    #('scaler',scaler),
    ('cutter', cutter), 
    ('offset_transformer',offset_transformer),
    ('linear_estimator', direct_linear_estimator)]
        
pipeline_direct_linear = Pipeline(steps) # define the pipeline object.

## Linear analytical method

In [ ]:
analytical_linear_estimator = AnalyticalLinearEstimator(omega_regression=True)

steps = [
    ('filter',lowpass_filter),
    #('scaler',scaler),
    ('cutter', cutter), 
    ('offset_transformer',offset_transformer),
    ('analytical_linear_estimator', analytical_linear_estimator)]
        
pipeline_analytical_linear = Pipeline(steps) # define the pipeline object.

## Quadratic direct

In [ ]:
direct_estimator = DirectEstimator(omega_regression=True, fit_method='derivation')
#direct_estimator = DirectEstimator(omega_regression=True, fit_method='integration')

steps = [
    ('filter',lowpass_filter),
    #('scaler',scaler),
    ('cutter', cutter), 
    ('offset_transformer',offset_transformer),
    ('direct_estimator', direct_estimator)]
        
pipeline_direct = Pipeline(steps) # define the pipeline object.

## Cubic

In [ ]:
estimator_cubic = EstimatorCubic()

steps = [
    ('filter',lowpass_filter),
    #('scaler',scaler),
    ('cutter', cutter), 
    ('offset_transformer',offset_transformer),
    ('direct_estimator_cubic', estimator_cubic)]
        
pipeline_cubic = Pipeline(steps) # define the pipeline object.

## Quadratic B & C

In [ ]:
estimator_quadratic_b_and_c = EstimatorQuadraticBandC()

steps = [
    ('filter',lowpass_filter),
    #('scaler',scaler),
    ('cutter', cutter), 
    ('offset_transformer',offset_transformer),
    ('direct_estimator_cubic', estimator_quadratic_b_and_c)]
        
pipeline_quadratic_b_and_c = Pipeline(steps) # define the pipeline object.

## Quadratic B (not C)

In [ ]:
estimator_quadratic_b = EstimatorQuadraticB()

steps = [
    ('filter',lowpass_filter),
    #('scaler',scaler),
    ('cutter', cutter), 
    ('offset_transformer',offset_transformer),
    ('direct_estimator_cubic', estimator_quadratic_b)]
        
pipeline_quadratic_b = Pipeline(steps) # define the pipeline object.

## Linear

In [ ]:
estimator_linear = EstimatorLinear()

steps = [
    ('filter',lowpass_filter),
    #('scaler',scaler),
    ('cutter', cutter), 
    ('offset_transformer',offset_transformer),
    ('direct_estimator_cubic', estimator_linear)]
        
pipeline_linear = Pipeline(steps) # define the pipeline object.

## Simplified Ikeda

In [ ]:
steps = [
    ('filter',lowpass_filter),
    #('scaler',scaler),
    ('cutter', cutter), 
    ('offset_transformer',offset_transformer),
    ]
        
preprocessor_ikeda = Pipeline(steps) # define the pipeline object.

In [ ]:
def fit_predict(db_run, pipeline):
    
    ascii_file = db_run.load()
    df_raw = ascii_file.channels
    
    df = signal_lab.mdl_to_evaluation.do_transforms(df=df_raw)
    
    #scaler = _pipline['scaler']
    #scaler.scale_factor = db_run.model.scale_factor
    
    df.rename(columns={'MA/Roll':'phi'}, inplace=True)
    
    _pipline_derivation = copy.deepcopy(pipeline)
    estimator_derivation = _pipline_derivation[-1]
    estimator_derivation.set_fit_method(fit_method='derivation')
    _pipline_derivation.fit(X=df)

    _pipline_integration = copy.deepcopy(pipeline)
    estimator_integration = _pipline_integration[-1]
    estimator_integration.set_fit_method(fit_method='integration')
    
    # Take the best of the derivation or integration approach:
    try:
        _pipline_integration.fit(X=df)
    except:
        estimator = estimator_derivation
        _pipline = _pipline_derivation
    else:
        if estimator_integration.score() > estimator_derivation.score():
            estimator = estimator_integration
            _pipline = _pipline_integration
        else:
            estimator = estimator_derivation
            _pipline = _pipline_derivation
    
    scale_factor=db_run.model.scale_factor
    meta_data = {
        'Volume':db_run.loading_condition.Volume/(scale_factor**3),
        'GM':db_run.loading_condition.gm/scale_factor,         
    }
    
    s = estimator.result_for_database(meta_data=meta_data)
    
    return s,_pipline, df
    
    

In [ ]:
def fit_predict_ikeda(db_run, pipeline, verify_input=True, limit_inputs=True):
    
    ascii_file = db_run.load()
    df_raw = ascii_file.channels
    
    df = signal_lab.mdl_to_evaluation.do_transforms(df=df_raw)
        
    _pipline = copy.deepcopy(pipeline)
    
    #scaler = _pipline['scaler']
    #scaler.scale_factor = db_run.model.scale_factor
    
    df.rename(columns={'MA/Roll':'phi'}, inplace=True)
    
    # Fit:
    _pipline.fit(X=df)
    X = _pipline.transform(X=df)
    
    # Predict:
    scale_factor=db_run.model.scale_factor
    lpp=db_run.ship.lpp/scale_factor
    TA=db_run.loading_condition.TA/scale_factor 
    TF=db_run.loading_condition.TF/scale_factor
    beam=db_run.ship.beam/scale_factor
    BKL=db_run.ship.BKL/scale_factor
    BKB=db_run.ship.BKB/scale_factor
    A0=db_run.loading_condition.A0
    kg=db_run.loading_condition.kg/scale_factor
    Volume=db_run.loading_condition.Volume/(scale_factor**3)
    gm=db_run.loading_condition.gm/scale_factor 
    V=db_run.ship_speed*1.852/3.6/np.sqrt(scale_factor)  #[m/s]
    rho=1000
    g=9.81
        
    estimator = IkedaQuadraticEstimator(lpp=lpp, TA=TA, TF=TF, beam=beam, BKL=BKL, BKB=BKB, A0=A0, 
                               kg=kg, Volume=Volume, gm=gm, rho=rho, g=g, V=V, 
                                        verify_input=verify_input, limit_inputs=limit_inputs)
    
    #estimator.fit(X=X)
    estimator.fit(X=X, alternative_bilge_keel=True, RdivB=0.08)  # this give better results.
    
    s = estimator.result_for_database()
    
    return s,_pipline, df

In [ ]:
roll_decay_tests = db.session.query(Run).filter(Run.test_type=='roll decay')

## Add roll decay

In [ ]:
#db_run = roll_decay_tests[0]
db_run = db.session.query(Run).get(int(6440))

s, pipeline, df = fit_predict(db_run=db_run, pipeline=pipeline_cubic)

In [ ]:
steps = [
    ('filter',lowpass_filter),
    ('scaler',scaler),
    ('cutter', cutter), 
]

preprosessor = Pipeline(steps=steps)
preprosessor['scaler'].scale_factor = db_run.model.scale_factor
preprosessor.fit(X=df)
X = preprosessor.transform(df)

In [ ]:
X.plot(y='phi1d')

In [ ]:
s

In [ ]:
#rolldecay_linear_db = RolldecayDirectCubic(run_id=db_run.id,**s)
#db.session.merge(rolldecay_linear_db)
#db.session.commit()

In [ ]:
def plot_pipeline(pipeline, df):
    df_ = df.copy()
    df_['n'] = n = np.arange(len(df_))
    pipeline = copy.deepcopy(pipeline)
    #pipeline['scaler'].scale_factor = db_run.model.scale_factor
    
    fig,axes = plt.subplots(nrows=len(pipeline))
    fig.set_size_inches(15,7)
    ax=axes[0]
    df_.plot(x='n', y='phi', ax=ax, label='raw')
    ax.grid(True)
    ax.legend()
    
    for step,ax in zip(pipeline[0:-1],axes[1:]):
       
        step.fit(X=df_)
        df_ = step.transform(X=df_)
    
        df_.plot(x='n', y='phi', label=str(step), ax=ax)
        ax.set_xlim(n[0], n[-1])
        ax.grid(True)
        ax.legend()
    
    fig,ax = plt.subplots()
    fig.set_size_inches(15,5)
    estimator = pipeline[-1]
    estimator.fit(X=df_)
    df_pred = estimator.predict(X=df_)
    estimator.plot_fit(ax=ax)
        

In [ ]:
#plot_pipeline(pipeline=pipeline_direct_linear,df=df)

In [ ]:
#o = pipeline_direct_linear['offset_transformer']
#o.X_zerocrossings()

In [ ]:
from datetime import datetime

roll_decay_tests = db.session.query(Run).filter(Run.test_type=='roll decay')

pipelines = [
    #(pipeline_direct_linear,RolldecayLinear),
    #(pipeline_direct,RolldecayDirect),
    (pipeline_cubic,RolldecayCubic),
    (pipeline_quadratic_b,RolldecayQuadraticB),
    (pipeline_quadratic_b_and_c,RolldecayQuadraticBandC),
    (pipeline_linear,RolldecayLinearB),
    
    #(pipeline_norwegian,RolldecayNorwegian),
    #(pipeline_direct_improved,RolldecayDirectImproved),
    #(pipeline_analytical_linear,RolldecayLinearAnalytical),
]

skipped=[]

for db_run in roll_decay_tests:
    
    if db_run.ship_speed is None:
        db_run.ship_speed=0
    
    for item in pipelines:  
        p = item[0]
        DBClass = item[1]
        
        if db.session.query(DBClass).get(db_run.id):
            continue  # This has already been handled
        
        print('%s run id:%i' % (datetime.now(),db_run.id))
        
        try:
            s, pipeline, df = fit_predict(db_run=db_run, pipeline=p)
        except Exception as e:
            skipped.append(db_run)
            warnings.warn(str(e), Warning)
            continue            
        else:
        
            rolldecay_db = DBClass(run_id=db_run.id,**s)
            db.session.merge(rolldecay_db)
            db.session.commit()
        
        

In [ ]:
for db_run in skipped:
    df = db_run.load().channels
    df = signal_lab.mdl_to_evaluation.do_transforms(df=df)
    df.rename(columns={'MA/Roll':'phi'}, inplace=True)
    
    df['phi_deg'] = np.rad2deg(df['phi'])
    df.plot(y='phi_deg')

In [ ]:
skipped=[]
pipelines = [
    (preprocessor_ikeda,RolldecaySimplifiedIkeda),
    ]

for db_run in roll_decay_tests:
    
    if db_run.ship_speed is None:
        db_run.ship_speed=0
    
    for item in pipelines:  
        p = item[0]
        DBClass = item[1]
        
        if db.session.query(DBClass).get(db_run.id):
            continue  # This has already been handled
        

        try:
            s, pipeline, df = fit_predict_ikeda(db_run=db_run, pipeline=p)
        except Exception as e:
            warnings.warn(str(e), Warning)
            
            if e is SimplifiedIkedaInputError:
                continue
            elif e is IkedaEstimatorFitError:
                continue
            else:
                skipped.append(db_run)
                continue
        else:
            rolldecay_db = DBClass(run_id=db_run.id,**s)
            db.session.merge(rolldecay_db)
            db.session.commit()

In [ ]:
skipped=[]
pipelines = [
    (preprocessor_ikeda,RolldecaySimplifiedIkedaUnlimited),
    ]

for db_run in roll_decay_tests:
    
    if db_run.ship_speed is None:
        db_run.ship_speed=0
    
    for item in pipelines:  
        p = item[0]
        DBClass = item[1]
        
        if db.session.query(DBClass).get(db_run.id):
            continue  # This has already been handled
        

        try:
            s, pipeline, df = fit_predict_ikeda(db_run=db_run, pipeline=p, verify_input=False, limit_inputs=False)  #Unlimited!!!
        except Exception as e:
            warnings.warn(str(e), Warning)
            
            if e is SimplifiedIkedaInputError:
                continue
            elif e is IkedaEstimatorFitError:
                continue
            else:
                skipped.append(db_run)
                continue
        else:
            rolldecay_db = DBClass(run_id=db_run.id,**s)
            db.session.merge(rolldecay_db)
            db.session.commit()

In [ ]:
db_run

In [ ]:
for db_run in skipped:
    df = db_run.load().channels
    df = signal_lab.mdl_to_evaluation.do_transforms(df=df)
    df.rename(columns={'MA/Roll':'phi'}, inplace=True)
    
    df['phi_deg'] = np.rad2deg(df['phi'])
    df.plot(y='phi_deg')
    
    

In [ ]:
df = db_run.load().channels
df = signal_lab.mdl_to_evaluation.do_transforms(df=df)
df.rename(columns={'MA/Roll':'phi'}, inplace=True)

df.plot(y='phi')

In [ ]:
df = db_run.load().channels
df = signal_lab.mdl_to_evaluation.do_transforms(df=df)
df.rename(columns={'MA/Roll':'phi'}, inplace=True)

df.plot(y='phi')

In [ ]:
steps = [
    ('filter',lowpass_filter),
    ('scaler',scaler),
    ('cutter', cutter), 
    #('offset_transformer',offset_transformer),
]    
preprocessor = Pipeline(steps) # define the pipeline object.
preprocessor.fit(df)
X = preprosessor.transform(df)

steps = [
    ('filter',lowpass_filter),
    ('scaler',scaler),
    #('cutter', cutter), 
    #('offset_transformer',offset_transformer),
]    
preprocessor2 = Pipeline(steps) # define the pipeline object.
preprocessor2.fit(df)
X2 = preprocessor2.transform(df)


fig,ax=plt.subplots()
X2.plot(y='phi',ax=ax)
X.plot(y='phi',ax=ax)


In [ ]:
8.5/25-0.2